In [ ]:
'''GET FINANCIAL DATA FROM FINNHUB API AND SAVE TO CSV FILES'''

import finnhub
import pandas as pd
import os
API_KEY = "d1vu9e1r01qmbi8q6n30d1vu9e1r01qmbi8q6n3g"

finnhub_client = finnhub.Client(api_key=API_KEY)

ticker = 'GOOGL'
# tickers = ['BEN', 'KMB', 'PITA', 'OMI', 'OCGN', 'NVT', 'BTSG', 'CNH', 'CHD', 'AVTR', 'RBC', 'MRNA', 'BFLY', 'GOOGL', 'AAPL', 'MSFT', 'AMZN', 'TSLA', 'META', 'NFLX']

tickers = ['FLWS',
 'VCXA',
 'TXG',
 'YI',
 'YQ',
 'TURN',
 'ATNF',
 'SRCE',
 'DIBS',
 'XXII',
 'MASK',
 'QFIN',
 'KRKR',
 'SCWO',
 'DDD',
 'MMM',
 'FDMT',
 'COE',
 'FEAM',
 'SXTP',
 'JEM',
 'ETNB',
 'EGHT',
 'MASS',
 'JFU',
 'ASPCR',
 'AMRK',
 'AOS',
 'ATEN',
 'AZ',
 'AAM',
 'AAON',
 'AIR',
 'ABM',
 'ABVC',
 'ACIU',
 'ACLO',
 'ACCS',
 'ACIW',
 'ACMR',
 'ACNB',
 'BRR',
 'ACR',
 'ACVA',
 'ADT',
 'ADTN',
 'ACM',
 'AEON',
 'AES',
 'LIDR']
#freq= 'quarterly' or 'annual'
#report_type = 'bs' for balance sheet, 'cf' for cash flow, 'ic' for income statement
def getFinancialData(ticker, freq='quarterly', report_type=['bs','cf','ic'],finnhub_client=finnhub_client):
    report_type_mappings = {
        'bs': 'balanceSheet',
        'cf': 'cashFlow',
        'ic': 'incomeStatement'
    }

    folder_path = f'./Data/finnhubDownloads/{ticker}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Get market capitalization
    # market_cap = finnhub_client.company_profile2(symbol=ticker)['marketCapitalization']

    # Get financials reported
    financials_lst = finnhub_client.financials_reported(symbol=ticker, freq=freq)['data']
    num_of_entries = len(financials_lst)
    for r in report_type:
        report_path = f"{folder_path}/{report_type_mappings[r]}"
        if not os.path.exists(report_path):
            os.makedirs(f"{report_path}")
            
        report_df = pd.DataFrame()
        for n in range(0, num_of_entries):
            financials_dict = financials_lst[n]
            financials_form = financials_dict['form']
            financials_startDate = financials_dict['startDate']
            financials_quarter = financials_dict['quarter']
            financials_endDate = financials_dict['endDate']
            financials_filedDate = financials_dict['filedDate']

            temp_data_df = pd.DataFrame()
            temp_data_df['endDate'] = [financials_endDate]

            print(f"Ticker: {ticker}, Form: {financials_form}, Filed Date: {financials_filedDate}")

            # Get report entries concepts
            def getReport(report_type,temp_data_df=temp_data_df):
                financials_report_dict = financials_dict['report'][report_type]
                for concept_index in range(0, len(financials_report_dict)):
                    concept = financials_report_dict[concept_index]['label']
                    concept_value = financials_report_dict[concept_index]['value']
                    temp_data_df[concept] = [concept_value]
                return temp_data_df
            report_df_n = getReport(r, temp_data_df)
            report_df = pd.concat([report_df, report_df_n], ignore_index=True)
        
        report_df.to_csv(f"{report_path}/{ticker}-{report_type_mappings[r]}.csv", index=False)

for i in tickers:
    getFinancialData(i, freq='quarterly', report_type=['bs','cf','ic'])
# getFinancialData('AAPL', freq='quarterly', report_type=['bs','cf','ic'])



In [ ]:
'''EARNINGS CALENDAR'''

import requests
import datetime

API_KEY = "d1vu9e1r01qmbi8q6n30d1vu9e1r01qmbi8q6n3g"
today = str(datetime.date.today())
print(today)

params = {
    "from": "2025-08-07",
    "to": "2025-08-07",
    # "symbol": "FI",  # optional
    "token": API_KEY
}

response = requests.get("https://finnhub.io/api/v1/calendar/earnings", params=params)
data = response.json()
print(len(data['earningsCalendar']))

tickers = []
if "earningsCalendar" in data:
    for report in data["earningsCalendar"]:
        tickers.append(report['symbol'])
        print(f"Ticker: {report['symbol']}, Date: {report['date']}, EPS Estimate: {report['epsEstimate']}, Time: {report['hour']}")
else:
    print("No earnings data found.")
print(tickers)